# SBI experiments: the Two Moons toy model

In [ ]:
import sys
sys.path.append('..')

import torch
import matplotlib.pyplot as plt
from sbi import analysis as analysis
from sbi import utils as utils
from sbi.inference import SNPE, simulate_for_sbi
from sbi.utils.user_input_checks import (
    check_sbi_inputs,
    process_prior,
    process_simulator,
)

from sbisandbox.examples.twomoons import TwoMoonsToyModel

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
two_moons = TwoMoonsToyModel()

In [ ]:
# Check prior, return PyTorch prior.
prior, num_parameters, prior_returns_numpy = process_prior(two_moons.prior)

# Check simulator, returns PyTorch simulator able to simulate batches.
simulator = process_simulator(two_moons.simulator, prior, prior_returns_numpy)

# Consistency check after making ready for sbi.
check_sbi_inputs(simulator, prior)

Plotting:

In [ ]:
theta, x = simulate_for_sbi(simulator, proposal=prior, num_simulations=10000)
fig, (ax1, ax2) = plt.subplots(2, 1)
ax1.hist2d(theta[:, 0], theta[:, 1], bins=100)
ax2.hist2d(x[:, 0], x[:, 1], bins=100)
ax1.set_title(r'$\theta$ samples')
ax2.set_title(r'Data samples')
fig.tight_layout()
plt.show()

In [ ]:
inference = SNPE(prior=prior)
inference = inference.append_simulations(theta, x)

In [ ]:
density_estimator = inference.train()

In [ ]:
posterior = inference.build_posterior(density_estimator)

In [ ]:
x_0 = torch.zeros(2)
posterior_samples = posterior.sample((10000,), x=x_0)

# plot posterior samples
_ = analysis.pairplot(
    posterior_samples, 
    limits=[[-0.5, 0.5], [-0.5, 0.5]],
    labels=['$\\theta_1$', '$\\theta_2$'],
    figsize=(5, 5)
)